In [1]:
import torch
import numpy as np
import gradio as gr
import IPython

from TTS.api import TTS
from pprint import pprint
from IPython.display import Audio
from langchain_community.llms import Ollama
from optimum.bettertransformer import BetterTransformer
from langchain.callbacks.manager import CallbackManager
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
import pygame

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.mixer.init()

In [3]:
tts = TTS("tts_models/en/vctk/vits").to('cuda')

 > tts_models/en/vctk/vits is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > initialization of speaker-embedding layers.


In [4]:
class AudioPlaybackCallback(BaseCallbackHandler):
    def __init__(self):
        self.content = ""
        
    def on_llm_new_token(self, token: str, **kwargs):
        self.content += token
        if self.content.strip()[-1] in ['.', '?', '!', ':']:
            wav = tts.tts(self.content, speaker=tts.speakers[2], verbose=False)
            filename = "soundbites/"+ self.content[:12] + ".wav"
            tts.tts_to_file(text=self.content, speaker=tts.speakers[2], file_path=filename)
            
            if pygame.mixer.music.get_busy() == True:
                pygame.mixer.music.queue(filename)
            else:
                pygame.mixer.music.load(filename)
                pygame.mixer.music.play()
            print(self.content)
            self.content = ""

handler = AudioPlaybackCallback()

In [5]:
llm = Ollama(
    model="llama2",
    callbacks=[handler],
)

In [6]:
whisper_model_id = "distil-whisper/distil-medium.en"

whisper = AutoModelForSpeechSeq2Seq.from_pretrained(
    whisper_model_id, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True# , use_flash_attention_2=True
)
whisper.to("cuda")
# model = model.to_bettertransformer() # we are using optimum BetterTransformer since Flash Attention 2 isn't supported on Colab
processor = AutoProcessor.from_pretrained(whisper_model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
asr_pipe = pipeline(
    "automatic-speech-recognition",
    model=whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15, #long form transcription
    batch_size=16,
    torch_dtype=torch.float16,
    device='cuda',
)

In [8]:
def transcribe_streaming(stream, new_chunk):
    sr, y = new_chunk
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y
    return stream, pipe({"sampling_rate": sr, "raw": stream})["text"]


In [13]:
def transcribe_streaming(stream, new_chunk):
    sr, y = new_chunk
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y

    transcribed_stream = asr_pipe({"sampling_rate": sr, "raw": stream})["text"]
    return stream, transcribed_stream

def answer_questions(stream):
    if stream.strip().endswith('?'):
        llm(stream)
    else:
        llm("Say: Sorry, I can't help you with that")

with gr.Blocks() as demo:
    textbox = gr.Textbox()
    mic_transcribe = gr.Interface(
        title='My Audio Transcription App Powered by Distill Whisper',
        description="Start recording",
        fn=transcribe_streaming,
        inputs=["state", gr.Audio(sources="microphone", streaming=True)],
        outputs=["state", textbox],
        live=True,
    )
    button = gr.Button("Answer")
    button.click(answer_questions, textbox, None)

demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


/home/ismail/envs/hym/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 > Text splitted to sentences.
["The sky appears blue because of a phenomenon called Rayleigh scattering, which occurs when light travels through the Earth's atmosphere."]
 > Processing time: 0.1654205322265625
 > Real-time factor: 0.028266605204554426
 > Text splitted to sentences.
["The sky appears blue because of a phenomenon called Rayleigh scattering, which occurs when light travels through the Earth's atmosphere."]
 > Processing time: 0.17553949356079102
 > Real-time factor: 0.030602829166788758
 The sky appears blue because of a phenomenon called Rayleigh scattering, which occurs when light travels through the Earth's atmosphere.
 > Text splitted to sentences.
['Blue light has shorter wavelengths than other colors, so it scatters more easily and is visible from a greater distance.']
 > Processing time: 0.18590044975280762
 > Real-time factor: 0.03208441544340488
 > Text splitted to sentences.
['Blue light has shorter wavelengths than other colors, so it scatters more easily and 

 > Processing time: 0.17913603782653809
 > Real-time factor: 0.026016635275550405
 This is why the sky appears blue during the daytime, when the sunlight is scattered in all directions by the tiny molecules of gases in the atmosphere.
 > Text splitted to sentences.
['The reason why the sky appears blue at sunrise and sunset is due to a different phenomenon called Mie scattering.']
 > Processing time: 0.1715083122253418
 > Real-time factor: 0.03282320410853342
 > Text splitted to sentences.
['The reason why the sky appears blue at sunrise and sunset is due to a different phenomenon called Mie scattering.']
 > Processing time: 0.1668863296508789
 > Real-time factor: 0.0325903674436896


The reason why the sky appears blue at sunrise and sunset is due to a different phenomenon called Mie scattering.
 > Text splitted to sentences.
['This occurs when light travels through a layer of atmosphere that contains particles, such as dust or water droplets, that are larger than the wavelength of th

 > Processing time: 0.20679450035095215
 > Real-time factor: 0.026503177791886535
 In this case, the light scatters in all directions, but it appears more red and orange because these colors have longer wavelengths and are scattered less than blue and violet light.
 > Text splitted to sentences.
["It's also worth noting that the color of the sky can be affected by other factors, such as pollution, dust, and smoke, which can scatter light in different ways and change the appearance of the sky."]
 > Processing time: 0.24992728233337402
 > Real-time factor: 0.028064127431408868
 > Text splitted to sentences.
["It's also worth noting that the color of the sky can be affected by other factors, such as pollution, dust, and smoke, which can scatter light in different ways and change the appearance of the sky."]
 > Processing time: 0.2117013931274414
 > Real-time factor: 0.023865111035072002


It's also worth noting that the color of the sky can be affected by other factors, such as pollution,

...

 > Processing time: 0.2314145565032959
 > Real-time factor: 0.03527465829898293
 It can securely store and transfer data, enabling new use cases such as decentralized finance (DeFi) and non-fungible tokens (NFTs).
 > Text splitted to sentences.
['8.']
 > Processing time: 0.14329004287719727
 > Real-time factor: 0.1450930127407329
 > Text splitted to sentences.
['8.']
 > Processing time: 0.22840332984924316
 > Real-time factor: 0.22089006241999173

8.
 > Text splitted to sentences.
['Synthetic Biology:']
 > Processing time: 0.22388648986816406
 > Real-time factor: 0.14713570283717864
 > Text splitted to sentences.
['Synthetic Biology:']
 > Processing time: 0.20339536666870117
 > Real-time factor: 0.12507998201263
 Synthetic Biology:
 > Text splitted to sentences.
['This field involves the design and construction of new biological systems, such as microbes that can produce biofuels or clean up environmental pollutants.']
 > Processing time: 0.19318699836730957
 > Real-time factor: 0.0250

 > Processing time: 0.19899535179138184
 > Real-time factor: 0.0360795249555977
 > Text splitted to sentences.
['Synthetic biology has the potential to revolutionize various industries, from agriculture to pharmaceuticals.']
 > Processing time: 0.1881577968597412
 > Real-time factor: 0.03341128254056576
 Synthetic biology has the potential to revolutionize various industries, from agriculture to pharmaceuticals.
 > Text splitted to sentences.
['9.']
 > Processing time: 0.13103246688842773
 > Real-time factor: 0.13113951955745423
 > Text splitted to sentences.
['9.']
 > Processing time: 0.1121530532836914
 > Real-time factor: 0.1122446815951977

9.
 > Text splitted to sentences.
['Robotics:']
 > Processing time: 0.1190803050994873
 > Real-time factor: 0.1035378835742782
 > Text splitted to sentences.
['Robotics:']
 > Processing time: 0.1753709316253662
 > Real-time factor: 0.13360036768723482
 Robotics:
 > Text splitted to sentences.
['Advances in robotics are enabling the development o

 > Processing time: 0.1973130702972412
 > Real-time factor: 0.036387270842149814
 Robots are also being used in logistics and transportation, improving efficiency and reducing costs.
 > Text splitted to sentences.
['10.']
 > Processing time: 0.19170331954956055
 > Real-time factor: 0.18965623636341575
 > Text splitted to sentences.
['10.']
 > Processing time: 0.17923474311828613
 > Real-time factor: 0.18585995512406928

10.
 > Text splitted to sentences.
['Nanotechnology:']
 > Processing time: 0.11943244934082031
 > Real-time factor: 0.08427692997840143
 > Text splitted to sentences.
['Nanotechnology:']
 > Processing time: 0.18063855171203613
 > Real-time factor: 0.13178533831559014
 Nanotechnology:
 > Text splitted to sentences.
['Researchers are making breakthroughs in nanotechnology, enabling the development of new materials with unique properties.']
 > Processing time: 0.19725942611694336
 > Real-time factor: 0.03509982525725146
 > Text splitted to sentences.
['Researchers are maki

 These materials have potential applications in fields such as medicine, energy, and electronics.
 > Text splitted to sentences.
['These are just a few examples of the exciting developments underway in various fields.']
 > Processing time: 0.212266206741333
 > Real-time factor: 0.053293744974567236
 > Text splitted to sentences.
['These are just a few examples of the exciting developments underway in various fields.']
 > Processing time: 0.19959306716918945
 > Real-time factor: 0.0489699476041551


These are just a few examples of the exciting developments underway in various fields.
 > Text splitted to sentences.
['As technology continues to evolve at an unprecedented pace, we can expect even more innovative solutions to emerge in the future.']
 > Processing time: 0.21941041946411133
 > Real-time factor: 0.03326823460490466
 > Text splitted to sentences.
['As technology continues to evolve at an unprecedented pace, we can expect even more innovative solutions to emerge in the future.'

 > Processing time: 0.22778868675231934
 > Real-time factor: 0.0347835217651568
 As technology continues to evolve at an unprecedented pace, we can expect even more innovative solutions to emerge in the future.
